In [67]:
import tensorflow as tf
from pymagnitude import *
import os
import csv
from tqdm import tqdm
import logging
from collections import Counter
import pickle
import re
import regex
import logging
import numpy as np

TRAINING_DATAFILE = "/Users/zxq001/QNLI/train.tsv"

In [53]:
# load pretrained embedding
vectors = Magnitude("/Users/zxq001/glove.840B.300d.magnitude")

In [54]:
i = tf.keras.layers.Input(shape=(None,))
embeddings = tf.keras.layers.Embedding(len(vectors), vectors.dim)(i)
lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(vectors.dim, return_sequences=True))(embeddings)
maxpool = tf.keras.layers.GlobalMaxPooling1D()(lstm)
output = tf.keras.layers.Dense(1, activation="softmax")(maxpool)
model = tf.keras.Model(inputs=i, outputs=output)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 300)         658805100 
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 600)         1442400   
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 600)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 601       
Total params: 660,248,101
Trainable params: 660,248,101
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# trains and saves the model
def train():
    # load data
    questions, answers, labels = read_QNLI_dataset(TRAINING_DATAFILE)
    
    # embedding transformation
    X1 = vectors.query(questions)
    X2 = vectors.query(answers)
    y = labels
    
    # concatnate input vectors
    X = tf.concat([X1, X2, abs(X1-X2), tf.math.multiply(X1, X2)], 1)
    
    # create model
    i = tf.keras.layers.Input(shape=(None,))
    embeddings = tf.keras.layers.Embedding(len(vectors), vectors.dim)(i)
    lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(vectors.dim, return_sequences=True))(embeddings)
    maxpool = tf.keras.layers.GlobalMaxPooling1D()(lstm)
    output = tf.keras.layers.Dense(1, activation="softmax")(maxpool)
    model = tf.keras.Model(inputs=i, outputs=output)
    

In [56]:
# read QNLI dataset from GLUE benchmark
def read_QNLI_dataset(FILE_PATH):
    trainfile = open(FILE_PATH)
    trainfile = csv.reader(trainfile, delimiter='\t')
    questions_raw = []
    answers_raw = []
    labels = []
    # skip the header
    next(trainfile)
    for row in trainfile:
        questions_raw.append(row[1])
        answers_raw.append(row[2])
        if row[3] == "entailment":
            labels.append(1)
        else:
            labels.append(0)
    # seperate the questions and answers into words
    questions = []
    answers = []
    for question in questions_raw:
        questions.append(regex.findall(r"[^[:punct:] ]+|[[:punct:]]", question))
    for answer in answers_raw:
        answers.append(regex.findall(r"[^[:punct:] ]+|[[:punct:]]", answer))
    return questions, answers, labels

In [30]:
questions, answers, labels = read_QNLI_dataset(TRAINING_DATAFILE)

In [46]:
vectors.query(questions[0:3]).shape

(3, 17, 300)

In [58]:
X1 = vectors.query(questions)
X2 = vectors.query(answers)
y = labels

In [59]:
X1.shape

(103140, 60, 300)

In [60]:
X2.shape

(103140, 1761, 300)

In [61]:
abs(X1-X2)

ValueError: operands could not be broadcast together with shapes (103140,60,300) (103140,1761,300) 

In [79]:
len(np.amax(answers))

60

In [70]:
X2[32972].shape

(1761, 300)

In [73]:
len(answers[32972])

60

In [76]:
X2[1].shape

(1761, 300)

In [77]:
len(answers[1])

30